In [1]:
import warnings
warnings.filterwarnings('ignore') # We can suppress the warnings

In [2]:
spark

In [3]:
import pandas as pd
import numpy as np
from PIL import Image
from io import BytesIO
import re
# import tensorflow as tf
# import cv2

I will use the functionality of Image Datasource of spark to collect and process images in bytes, and then start processing the images for the neural network

In [4]:
images_folders = ['/CA1/Images/Ireland/','/CA1/Images/Honduran/']

In [5]:
images_rdd = spark.sparkContext.binaryFiles(','.join(images_folders))
# images_rdd = spark.sparkContext.binaryFiles('hdfs://172.24.144.178:9000/CA1/Images/Ireland/Ireland_001.jpeg')

In [6]:
images_rdd

org.apache.spark.api.java.JavaPairRDD@7aa32c3b

In [7]:
def extract_data(data):
    file_path, image_data = data
#     image = Image.open(BytesIO(image_data))
#     image_array = np.array(image)
    
    file_name = file_path.split('/')[-1]
    file_name_without_ext = file_name.split(".")[0]
    label,name = file_name_without_ext.split('_')[0], file_name_without_ext.split('_')[1]
    
    return name, label, image_data

In [8]:
imageDf = images_rdd.map(lambda x: extract_data(x)).toDF(["Name","Label","Data"])

In [9]:
pandasImagesDF = imageDf.toPandas()

In [10]:
IMG_SHAPE=225

In [11]:
def processImage(data, target_size=(IMG_SHAPE,IMG_SHAPE)):
    imgbytes = BytesIO(data)
    image = Image.open(imgbytes)
    resized_img = image.resize(target_size, Image.ANTIALIAS)
    with BytesIO() as output:
        resized_img.save(output, format="PNG")
        new_image = Image.open(output)

        array = np.asarray(new_image).reshape([target_size[0],target_size[1],3])
        return array

In [12]:
pandasImagesDF["Data"] = pandasImagesDF["Data"].apply(lambda x: processImage(x))

In [13]:
# pandasImagesDF

In [14]:
pandasImagesDF = pandasImagesDF.sort_values(by=["Name"])

In [15]:
pandasImagesDF.info()

<class 'pandas.core.frame.DataFrame'>
Index: 180 entries, 0 to 179
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Name    180 non-null    object
 1   Label   180 non-null    object
 2   Data    180 non-null    object
dtypes: object(3)
memory usage: 5.6+ KB


In [16]:
NCATEGORIES = len(pandasImagesDF["Label"].unique())

Importing from Keras functionality necessary to implement CNN

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten, Input
from tensorflow.keras import utils
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

2024-03-22 23:12:38.810188: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-22 23:12:39.523363: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Now we are to split the data using train_test_split function to use Kfold and improve the perfomance of the model

In [18]:
X_train, X_test, y_train, y_test = train_test_split(pandasImagesDF["Data"],pandasImagesDF["Label"],test_size=0.3, random_state=42)

In [19]:
encoder = OneHotEncoder(sparse_output=False)
y_train = np.array(y_train).reshape(-1, 1)
y_train = encoder.fit_transform(y_train)

y_test = np.array(y_test).reshape(-1,1)
y_test = encoder.fit_transform(y_test)

In [20]:
X_train = np.stack(X_train)
X_test = np.stack(X_test)

In [21]:
X_train.shape

(126, 225, 225, 3)

In [22]:
X_train = np.reshape(X_train,(X_train.shape[0],IMG_SHAPE,IMG_SHAPE,3)).astype(np.float32)
X_test = np.reshape(X_test,(X_test.shape[0],IMG_SHAPE,IMG_SHAPE,3)).astype(np.float32)

Because the pixels are from 0 to 255, We have to normalize the pixels

In [23]:
X_train/=255
X_test/=255

In [24]:
X_train.shape,X_test.shape

((126, 225, 225, 3), (54, 225, 225, 3))

In [25]:
y_train.shape, y_test.shape

((126, 2), (54, 2))

In [26]:
model = Sequential()
model.add(Input(shape=(IMG_SHAPE, IMG_SHAPE, 3)))  # 225x225 RGB images
model.add(Conv2D(32,kernel_size=(3,3),strides=(1,1),padding="valid", activation="relu"))
model.add(Conv2D(32,kernel_size=(3,3),activation="relu"))
model.add(MaxPool2D(3))
model.add(Conv2D(32, (3,3), activation="relu"))
model.add(Conv2D(32, (3,3), activation="relu"))
model.add(MaxPool2D(2))
model.add(Conv2D(32, (3,3), activation="relu"))
model.add(Conv2D(32, (3,3), activation="relu"))
model.add(MaxPool2D(2))
model.add(Conv2D(32, (3,3), activation="relu"))
model.add(Conv2D(32, (3,3), activation="relu"))
model.add(MaxPool2D(2))

model.add(Flatten())


model.add(Dense(128, activation="relu"))
model.add(Dense(NCATEGORIES,activation="sigmoid"))

model.compile(loss="binary_crossentropy", metrics=["accuracy"], optimizer="adam")

2024-03-22 23:12:40.696437: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-22 23:12:40.751300: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-22 23:12:40.751391: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-22 23:12:40.756279: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-22 23:12:40.756344: I external/local_xla/xla/stream_executor

In [27]:
model.fit(X_train, y_train, batch_size=50, epochs=40, verbose=1, validation_data=(X_test, y_test))

Epoch 1/40


I0000 00:00:1711149163.226763   13701 service.cc:145] XLA service 0x7f391c0127b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1711149163.226806   13701 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 2070 with Max-Q Design, Compute Capability 7.5
2024-03-22 23:12:43.277138: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-22 23:12:43.585572: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
2024-03-22 23:12:49.441029: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng4{k11=1} for conv (f32[50,32,73,73]{3,2,1,0}, u8[0]{0}) custom-call(f32[50,32,71,71]{3,2,1,0}, f32[32,32,3,3]{3,2,1,0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardInput", backend_config={"operation_queue_id":"0","wait_on_opera

2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - accuracy: 0.4800 - loss: 0.6932

I0000 00:00:1711149180.529490   13701 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2024-03-22 23:13:04.466944: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng4{k11=1} for conv (f32[26,32,73,73]{3,2,1,0}, u8[0]{0}) custom-call(f32[26,32,71,71]{3,2,1,0}, f32[32,32,3,3]{3,2,1,0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardInput", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0}} is taking a while...
2024-03-22 23:13:04.729214: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.262354757s
Trying algorithm eng4{k11=1} for conv (f32[26,32,73,73]{3,2,1,0}, u8[0]{0}) custom-call(f32[26,32,71,71]{3,2,1,0}, f32[32,32,3,3]{3,2,1,0}), window={size=3x3}, dim_labels=bf01_oi01->bf0

3/3 ━━━━━━━━━━━━━━━━━━━━ 33s 7s/step - accuracy: 0.4741 - loss: 0.6938 - val_accuracy: 0.4259 - val_loss: 0.6920
Epoch 2/40
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - accuracy: 0.4587 - loss: 0.6933 - val_accuracy: 0.4259 - val_loss: 0.6922
Epoch 3/40
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 0.5109 - loss: 0.6927 - val_accuracy: 0.4259 - val_loss: 0.6904
Epoch 4/40
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - accuracy: 0.5259 - loss: 0.6911 - val_accuracy: 0.4259 - val_loss: 0.6810
Epoch 5/40
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.5359 - loss: 0.6940 - val_accuracy: 0.4259 - val_loss: 0.6750
Epoch 6/40
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 0.5384 - loss: 0.6857 - val_accuracy: 0.4259 - val_loss: 0.6834
Epoch 7/40
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - accuracy: 0.5134 - loss: 0.6915 - val_accuracy: 0.4259 - val_loss: 0.6876
Epoch 8/40
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.5359 - loss: 0.6889 - val_accuracy: 0.4259 - val_loss: 0.6864
Epoch 9/40
3

In [28]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 223, 223, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 221, 221, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 73, 73, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 71, 71, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 69, 69, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 34, 34, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 30, 30, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 13, 13, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 11, 11, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 5, 5, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 800)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       102,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 505,256 (1.93 MB)

 Trainable params: 168,418 (657.88 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 336,838 (1.28 MB)

In [29]:
pred = model.predict(X_test[:25])

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


In [30]:
def comparePredictions(predictionsValues, expectedValues):
    correct = 0
    for x in range(len(predictionsValues)):
        if predictionsValues[x] == expectedValues[x]:
            correct+=1
    
    print(f'Correct predictions {correct}/{len(predictionsValues)}. Percent {correct/len(predictionsValues)}')

In [31]:
comparePredictions(np.argmax(pred, axis=1),np.argmax(y_test[:25], axis=1))

Correct predictions 12/25. Percent 0.48


### Trying Data Augmentation 

In [32]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [33]:
X_train, X_test, y_train, y_test = train_test_split(pandasImagesDF["Data"],pandasImagesDF["Label"],test_size=0.3, random_state=42)

In [34]:
encoder = OneHotEncoder(sparse_output=False)
y_train = np.array(y_train).reshape(-1, 1)
y_train = encoder.fit_transform(y_train)

y_test = np.array(y_test).reshape(-1,1)
y_test = encoder.fit_transform(y_test)

X_train = np.stack(X_train)
X_train = np.reshape(X_train,(X_train.shape[0],IMG_SHAPE,IMG_SHAPE,3)).astype(np.float32)
X_train/=255

X_test = np.stack(X_test)
X_test = np.reshape(X_test,(X_test.shape[0],IMG_SHAPE,IMG_SHAPE,3)).astype(np.float32)
X_test/=255

In [35]:
datagen = ImageDataGenerator(
    rotation_range=20,  # Random rotation between 0 and 20 degrees
    width_shift_range=0.1,  # Randomly shift the width by up to 10%
    height_shift_range=0.1,  # Randomly shift the height by up to 10%
    shear_range=0.2,  # Shear intensity (shear angle in counter-clockwise direction in radians)
    zoom_range=0.2,  # Randomly zoom in by up to 20%
    horizontal_flip=True,  # Randomly flip images horizontally
    fill_mode='nearest'  # Fill mode for points outside the input boundaries
)

In [36]:
print("Shape of X_train:", X_train.shape)
print("Type of X_train:", type(X_train))
print("Shape of y_train:", y_train.shape)
print("Type of y_train:", type(y_train))

Shape of X_train: (126, 225, 225, 3)
Type of X_train: <class 'numpy.ndarray'>
Shape of y_train: (126, 2)
Type of y_train: <class 'numpy.ndarray'>


In [37]:
#augmented_generator = datagen.flow(np.array(X_train), np.array(y_train), batch_size=32)

In [38]:
#augmented_generator

In [39]:
# # Get a batch of augmented data
# images, labels = next(augmented_generator)

# # Visualize the augmented images
# plt.figure(figsize=(10, 10))
# for i in range(9):
#     plt.subplot(3, 3, i + 1)
#     plt.imshow(images[i])
#     plt.title(f'Label: {labels[i]}')
#     plt.axis('off')
# plt.show()

In [40]:
from tensorflow.keras.layers import RandomContrast, RandomBrightness, RandomRotation, RandomFlip

In [41]:
dataAugmented = Sequential()
dataAugmented.add(RandomContrast(0.7))
dataAugmented.add(RandomBrightness(0.7))
dataAugmented.add(RandomRotation(0.4))
dataAugmented.add(RandomFlip())


In [42]:
modelAugmented = Sequential()
modelAugmented.add(Input(shape=(IMG_SHAPE, IMG_SHAPE, 3)))  # 225x225 RGB images
modelAugmented.add(dataAugmented)
modelAugmented.add(Conv2D(32,kernel_size=(3,3),strides=(1,1),padding="valid", activation="relu"))
modelAugmented.add(Conv2D(32,kernel_size=(3,3),activation="relu"))
modelAugmented.add(MaxPool2D(3))
modelAugmented.add(Conv2D(32, (3,3), activation="relu"))
modelAugmented.add(Conv2D(32, (3,3), activation="relu"))
modelAugmented.add(MaxPool2D(2))
modelAugmented.add(Conv2D(32, (3,3), activation="relu"))
modelAugmented.add(Conv2D(32, (3,3), activation="relu"))
modelAugmented.add(MaxPool2D(2))
modelAugmented.add(Conv2D(32, (3,3), activation="relu"))
modelAugmented.add(Conv2D(32, (3,3), activation="relu"))
modelAugmented.add(MaxPool2D(2))

modelAugmented.add(Flatten())


modelAugmented.add(Dense(128, activation="relu"))
modelAugmented.add(Dense(NCATEGORIES,activation="sigmoid"))

modelAugmented.compile(loss="binary_crossentropy", metrics=["accuracy"], optimizer="adam")

In [43]:
X_train.shape,X_test.shape, y_train.shape, y_test.shape

((126, 225, 225, 3), (54, 225, 225, 3), (126, 2), (54, 2))

In [44]:
modelAugmented.fit(X_train, y_train, batch_size=32, epochs=30, verbose=1, validation_data=(X_test, y_test))

Epoch 1/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.4957 - loss: 1.5276 - val_accuracy: 0.5741 - val_loss: 0.6925
Epoch 2/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - accuracy: 0.4590 - loss: 0.7867 - val_accuracy: 0.4259 - val_loss: 0.6936
Epoch 3/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - accuracy: 0.5420 - loss: 0.7086 - val_accuracy: 0.4259 - val_loss: 0.6936
Epoch 4/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - accuracy: 0.4469 - loss: 0.7032 - val_accuracy: 0.4259 - val_loss: 0.6939
Epoch 5/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - accuracy: 0.6220 - loss: 0.6955 - val_accuracy: 0.4259 - val_loss: 0.6944
Epoch 6/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - accuracy: 0.5314 - loss: 0.6993 - val_accuracy: 0.4259 - val_loss: 0.6949
Epoch 7/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.5200 - loss: 0.7107 - val_accuracy: 0.4259 - val_loss: 0.6951
Epoch 8/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - accuracy: 0.5606 - loss: 0.6845 - val_accuracy: 0.4259 - val_loss: 0.6953
Ep

In [45]:
modelAugmented.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_1 (Sequential)       │ (None, 225, 225, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 223, 223, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 221, 221, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 73, 73, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 71, 71, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 69, 69, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 34, 34, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 30, 30, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 13, 13, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 11, 11, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 5, 5, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 800)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       102,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 505,256 (1.93 MB)

 Trainable params: 168,418 (657.88 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 336,838 (1.28 MB)

In [46]:
pred = modelAugmented.predict(X_test[:25])

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 676ms/step


In [47]:
comparePredictions(np.argmax(pred, axis=1),np.argmax(y_test[:25], axis=1))

Correct predictions 11/25. Percent 0.44
